# The Analytics Edge of Amusement Park Souvenir Price Optimization

Both binary variables and non-linear functions


#gurobipy
The Gurobi Optimizer is a mathematical optimization software library for solving mixed-integer linear and quadratic optimization problems.

In [2]:
!pip install gurobipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 32.4 MB/s eta 0:00:00


# load require packages

In [3]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import plotly.express as px

### Lists

In [4]:
# Be careful with Python indices!
# The range function range(a,b) creates a range of integers starting at <a> but ending at <b-1>
weeks = list(range(157,165))
print("weeks (t) ", weeks)
# Also note here that the first element of a list has index 0. The second element has index 1, and so on...
w1 = weeks[0] #denotes the number of the first week in our planning horizon
print(w1)
w2 = weeks[1] #denotes the number of the second week in our planning horizon
print(w2)

weeks (t)  [157, 158, 159, 160, 161, 162, 163, 164]
157
158


### Parameters

In [5]:
#From orginal price / sales(demand) data

# Variable: 𝑆𝑒𝑎𝑠𝑜𝑛X. Year divided int 13 seasons / 4 weeks per season (categorical variable)
# Variable: Price,  at time 𝑝_𝑡, 𝑝_(𝑡−1), 𝑝_(𝑡−2)
# Linear regression developed demand model (Constrain 1)
# Demand is characterized by our linearly additive model

# Store model parameters
intercept = 1.978242858
# causal factors
p_coeff = -2.809634145
p1_coeff = 0.963410728
p2_coeff = 0.759639170
# time series factor
season_coeff = {1: 0, 2: -0.562046910, 3: 0.087545274,  4: -0.402637480, 5: -0.027326010, 6:  0.004349885,
               7: -0.036102297, 8: -0.069280527, 9: 0.160276197, 10: 1.104208897, 11: 1.122711287, 12: 1.176802194, 13: 0.947945548}

In [6]:
# Here we create a dictionary that associates a season with each week in the planning horizon
season = {}
for w in weeks:
    season[w] = np.ceil((w % 52) / 4)

print(season)

{157: 1.0, 158: 1.0, 159: 1.0, 160: 1.0, 161: 2.0, 162: 2.0, 163: 2.0, 164: 2.0}


## Model 1 - no price ladder or business rules
#### DECISION VARIABLES:  prices in weeks 157 – 164  and demand in weeks 157 – 164
######  Prices; 𝑝_𝑡  for  t = 157, … , 164   and   𝑑_𝑡  for  t = 157, … , 164
######  Demand; follows linearly additive model:
#### MAXIMIZE 𝑅𝑒𝑣𝑒𝑛𝑢𝑒= 𝑝157𝑑157 + 𝑝158𝑑158 + 𝑝159𝑑159 + ... + 𝑝164
#### Objective function,  sales revenue in next quarter (weeks 157 – 164)

1.   List item
2.   List item


#### SUBJECT TO CONSTRAINTS:
#### Demand is characterized by our linearly additive model:



In [9]:
# Create Gurobi model object - repository for all objects to be used in the model
mod1 = gp.Model ("price_model_1")

In [10]:
# Define decision variables
p = mod1.addVars(weeks, ub = 1)

In [11]:
p

{157: <gurobi.Var *Awaiting Model Update*>,
 158: <gurobi.Var *Awaiting Model Update*>,
 159: <gurobi.Var *Awaiting Model Update*>,
 160: <gurobi.Var *Awaiting Model Update*>,
 161: <gurobi.Var *Awaiting Model Update*>,
 162: <gurobi.Var *Awaiting Model Update*>,
 163: <gurobi.Var *Awaiting Model Update*>,
 164: <gurobi.Var *Awaiting Model Update*>}

In [13]:
# Set objective function
# Example first 2 lines fully written out,
# Short cut for writing out each line
#  after the first 2 weeks, the remaining weeks[2:], are developed using the sum() and a for loop  (weeks 159 on...)
#    price at week(wx) * demand (from linear demand fucntion at weekx)
obj_fn = mod1.setObjective(p[w1] * (intercept + p_coeff*p[w1] + p1_coeff*1 + p2_coeff*1 + season_coeff[season[w1]]) +
                           p[w2] * (intercept + p_coeff*p[w2] + p1_coeff*p[w1] + p2_coeff*1 + season_coeff[season[w2]]) +
                           sum(p[w] * (intercept + p_coeff*p[w] + p1_coeff*p[w-1] + p2_coeff*p[w-2] + season_coeff[season[w]]) for w in weeks[2:]),
                          GRB.MAXIMIZE)

In [14]:
mod1.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 0 rows, 8 columns and 0 nonzeros
Model fingerprint: 0x5e566187
Model has 21 quadratic objective terms
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 4e+00]
  QObjective range [2e+00, 6e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.01s
Presolved: 0 rows, 8 columns, 0 nonzeros
Presolved model has 21 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 7
 AA' NZ     : 2.100e+01
 Factor NZ  : 2.800e+01
 Factor Ops : 1.400e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -1.43138109e+05  1.691

In [15]:
p[157] # to see 1st prediction

<gurobi.Var C0 (value 0.9317730949053973)>

In [16]:
p[158] # to see 2nd prediction

<gurobi.Var C1 (value 0.907790044478695)>

In [17]:
# to see all predictions and save to a dataframe
df1 = pd.DataFrame(data = None, index = weeks, columns = ["price"])
for w in weeks:
    df1.loc[w,"price"] = p[w].x
df1

,price
157,0.931773
158,0.90779
159,0.868855
160,0.827353
161,0.690618
162,0.630388
163,0.526768
164,0.427557


In [18]:
fig = px.line(df1, x=df1.index, y='price', markers=True)
fig.update_layout(plot_bgcolor= "white", xaxis_title= "week")
fig.update_traces(line_color= "red")
fig.show()

## Model 2 - no business rules + price ladder

In [19]:
p_ladder = [1.0, 0.95, 0.85, 0.75, 0.60, 0.50] #create a list for price ladder

In [20]:
# Create model object
mod2 = gp.Model ("price_model_2")

In [21]:
# Define decision variables which includes ladder constraint
p = mod2.addVars(weeks)
x = mod2.addVars(weeks, p_ladder, vtype= GRB.BINARY)

In [22]:
# Set objective function
obj_fn = mod2.setObjective(p[w1] * (intercept + p_coeff*p[w1] + p1_coeff*1 + p2_coeff*1 + season_coeff[season[w1]]) +
                           p[w2] * (intercept + p_coeff*p[w2] + p1_coeff*p[w1] + p2_coeff*1 + season_coeff[season[w2]]) +
                           sum(p[w] * (intercept + p_coeff*p[w] + p1_coeff*p[w-1] + p2_coeff*p[w-2] + season_coeff[season[w]]) for w in weeks[2:]),
                          GRB.MAXIMIZE)

In [23]:
# Select price ladder value
constr_select_ladder = mod2.addConstrs(sum(x[w,k] for k in p_ladder) == 1 for w in weeks)
# Select price
constr_select_price = mod2.addConstrs(p[w] == sum(k * x[w,k] for k in p_ladder) for w in weeks)

In [24]:
mod2.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 16 rows, 56 columns and 104 nonzeros
Model fingerprint: 0xfbc9a1f9
Model has 21 quadratic objective terms
Variable types: 8 continuous, 48 integer (48 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+00]
  Objective range  [1e+00, 4e+00]
  QObjective range [2e+00, 6e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 5.0955328
Presolve removed 0 rows and 8 columns
Presolve time: 0.00s
Presolved: 16 rows, 48 columns, 88 nonzeros
Presolved model has 21 quadratic objective terms
Variable types: 8 continuous, 40 integer (40 binary)

Root relaxation: objective 6.242180e+00, 21 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective 

In [25]:
df2 = pd.DataFrame(data = None, index = weeks, columns = ["price"])
for w in weeks:
    df2.loc[w,"price"] = p[w].x
df2

,price
157,0.95
158,0.95
159,0.85
160,0.85
161,0.75
162,0.6
163,0.5
164,0.5


In [26]:
fig = px.line(df2, x=df2.index, y='price', markers=True)
fig.update_layout(plot_bgcolor= "white", xaxis_title= "week")
fig.update_traces(line_color= "red")
fig.show()